In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
import re
from sklearn.preprocessing import LabelEncoder

# Data Preprocessing

Firstly we split x and y of location .

In [ ]:
# at first we get the data set using pandas
Crime = pd.read_csv("/content/Raw_Dataset.csv" , encoding='windows-1254')
new_dataset = pd.DataFrame()
Crime.head()


ParserError: ignored

Here we are going to work on "OFFENSE_DESCRIPTION" at first we fill in the null data , then we eliminate " ?.!/;:" and finally we convert it to lower case.

In [ ]:
# Get Description And Cleaning & Convert To Lower Case
Description=Crime['OFFENSE_DESCRIPTION']

for key in Description.isnull():
    if key == True:
        print("NULL")

des=[]
for letter in Description:
    for char in letter:
        if char in " ?.!/;:":
           letter.replace(char,'')
    des.append(letter.lower().strip().replace(' - ','-'))
new_dataset["OFFENSE_DESCRIPTION"] = pd.DataFrame(des)

we maintain the "DISTRICT" , "DAY_OF_WEEK"  , "YEAR" , "HOUR" and "MONTH" features without any change .

In [ ]:
new_dataset["DISTRICT"] = Crime["DISTRICT"]
new_dataset["DAY_OF_WEEK"] = LabelEncoder().fit_transform(Crime["DAY_OF_WEEK"]) 
new_dataset["MONTH"] = Crime["MONTH"]
new_dataset["YEAR"] = Crime["YEAR"]
new_dataset["HOUR"] = Crime["HOUR"]

here we split the date and time of occurrence .

In [ ]:
OCCURRED_DATE=Crime["OCCURRED_ON_DATE"]
date=[]
time=[]
for i in OCCURRED_DATE:
    date.append(i.split(" ")[0])
    time.append(i.split(" ")[1])

In [ ]:
# our time consists of hour , minute and second but we don not need to 
# know the sceond so we eliminate it and maintain hour and minute 
new_time=[]
for t in time :
    split_timwe=t.split(':')
    hours=split_timwe[0]
    minutes=split_timwe[1]
    new_time.append(hours+":"+minutes)
# we alse change the form of our dates 
desired_date = '2018-08-30'
desired_list = [x for x in date if x >= desired_date]

In [ ]:
new_dataset["date"] = pd.DataFrame(desired_list)
new_dataset["time"] = pd.DataFrame(new_time)

In [ ]:
# Get UCR PART - And Encoding Them 
UCR_PART=Crime["UCR_PART"]
for item in range(len(Crime)) :
  if pd.isna(Crime["UCR_PART"][item]) :
    Crime["UCR_PART"].iloc[item] = "Part zero"

new_dataset["UCR_PART"] = LabelEncoder().fit_transform(Crime["UCR_PART"]) 

In [ ]:
# here we work on locataion . we add two columns for location_x and location_y 
# first we
def remove(text) :
    xandy = []
    x = text
    y = re.findall(r'\d+\.\d', x)
    for i in y:
        xandy.append(i)
    return xandy[0] , xandy[1]

In [ ]:
x = []
y = []

for item in range(len(Crime)) :
  if not pd.isna(Crime['Location'][item]) :
        x_location , y_location = remove(Crime['Location'][item])
        x.append(x_location)
        y.append(y_location)
       
new_dataset["X_Location"] = pd.DataFrame(x)
new_dataset["Y_Location"] = pd.DataFrame(y)                

in this section we have to add other columns that dont need preprocessing 
here the features are "OFFENSE_CODE_GROUP" , "STREET" , "Lat" , "Long"

In [ ]:
# we have to convert "OFFENSE_CODE_GROUP" to numerical . in the next step we wnt use thease data in our ML models

new_dataset["OFFENSE_CODE_GROUP"] = LabelEncoder().fit_transform(Crime["OFFENSE_CODE_GROUP"]) 

new_dataset["STREET"] = LabelEncoder().fit_transform(Crime["STREET"]) 

new_dataset["Lat"] = Crime["Lat"]

new_dataset["Long"] = Crime["Long"]


In [ ]:
# in this section we are going to fill in the null sectors of "SHOOTING" feature . we have approimately 324000 data 
# but only nearly 1700 of SHOOTING sectors have valeu , however all of them are "Y"  so maybe the outhers are "N" ,so 
# we consider them as "N" .
for item in range(len(Crime)) :
    if pd.isna(Crime["SHOOTING"][item]) :
        Crime["SHOOTING"].iloc[item] = "N"
new_dataset["SHOOTING"] = LabelEncoder().fit_transform(Crime["SHOOTING"]) 

# now we have to transform "Y" and "N" to 0 and 1 respectively by LabelEncoding in sklearn library .


In [ ]:
new_dataset["OFFENSE_CODE_GROUP_WITHOUT_ENCODING"] = Crime["OFFENSE_CODE_GROUP"]
new_dataset["STREET_WITHOUT_ENCODING"] = Crime["STREET"]

# Data Cleaning
here we are going to drop null data .

In [ ]:
final_dataset = new_dataset.dropna(subset= ["OFFENSE_CODE_GROUP"  , "MONTH" , "DAY_OF_WEEK" , "HOUR" , "X_Location", "Y_Location" ]) 

In [ ]:
final_dataset.to_csv("new_dataset.csv")

In [ ]:
duplicated = Crime["OFFENSE_CODE_GROUP"].duplicated()
data = [Crime["OFFENSE_CODE_GROUP"][i] for i in range(len(Crime["OFFENSE_CODE_GROUP"])) if duplicated[i] == False ]

data

In [ ]:
duplicated_day = Crime["DAY_OF_WEEK"].duplicated()
days = [Crime["DAY_OF_WEEK"][i] for i in range(len(Crime["DAY_OF_WEEK"])) if duplicated_day[i] == False ]
days

In [ ]:
Crime.info()